In [1]:
from fishsense_api_sdk.client import Client
from fishsense_api_sdk.models.image import Image
from pathlib import Path
import asyncio
from typing import List
import numpy as np
import rawpy
import matplotlib.pyplot as plt
from skimage.util import img_as_float, img_as_ubyte
import cv2
import math
from tqdm.notebook import tqdm
from skimage.exposure import adjust_gamma, equalize_adapthist
from synology_api.filestation import FileStation
from fishsense_data_processing_workflow_worker.config import settings

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 349

NAS_HOST = "e4e-nas.ucsd.edu"
NAS_PORT = 6021

In [3]:
DATA_FOLDER = (Path("../data") / "REEF" / "data").absolute()
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

DATA_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [4]:
client = Client(URL)

In [5]:
filestation = FileStation(NAS_HOST, NAS_PORT, settings.e4e_nas.username, settings.e4e_nas.password, secure=True, cert_verify=False)

User logged in, new session started!


In [6]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=349, name='031424_Alligator0_FSL06', path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL06/031424_Alligator0_FSL06', dive_datetime=datetime.datetime(2024, 3, 14, 10, 53, 15, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=6, dive_slate_id=None)

In [7]:
camera = await client.cameras.get(dive.camera_id)

camera

Camera(id=6, serial_number='BJ6C67987', name='FSL-06')

In [8]:
camera_intrinsics = await client.cameras.get_intrinsics(dive.camera_id)

camera_intrinsics

In [9]:
clusters = await client.images.get_clusters(dive_id=DIVE_ID)

len(clusters), len(clusters[0].image_ids), clusters[0].image_ids[0]

(50, 2, 102533)

In [10]:
cluster_images: List[List[Image]] = []

for cluster in clusters:
    cluster_images.append(await asyncio.gather(*[client.images.get(image_id=i) for i in cluster.image_ids]))

sum(len(c) for c in cluster_images), len(cluster_images), len(cluster_images[0]), cluster_images[0][0]

(174,
 50,
 2,
 Image(id=102533, path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL06/031424_Alligator0_FSL06/P3140114.ORF', taken_datetime=datetime.datetime(2024, 3, 14, 10, 18, 50, tzinfo=TzInfo(0)), checksum='6e07c6e2d810e825cb3f398a73a9e9a4', is_canonical=True, dive_id=349, camera_id=6))

In [11]:
def process_raw(image_path: Path):
    with image_path.open("rb") as f:
        with rawpy.imread(f) as raw:
            img = img_as_float(
                raw.postprocess(
                    gamma=(1, 1),
                    no_auto_bright=True,
                    use_camera_wb=True,
                    output_bps=16,
                    user_flip=0,
                )
            )

            hsv = cv2.cvtColor(img_as_ubyte(img), cv2.COLOR_BGR2HSV)
            _, _, val = cv2.split(hsv)

            mid = 20
            mean = np.mean(val)
            meanLog = math.log(mean)
            midLog = math.log(mid * 255)
            gamma = midLog / meanLog
            gamma = 1 / gamma

            img = adjust_gamma(img, gamma=gamma)

            img = equalize_adapthist(img)

            return img_as_ubyte(img[:, :, ::-1])

In [12]:
def rectify(img: np.ndarray, camera_matrix: np.ndarray, distortion_coefficients: np.ndarray) -> np.ndarray:
    return cv2.undistort(
        img,
        camera_matrix,
        distortion_coefficients,
    )

In [13]:
for cluster in tqdm(cluster_images):
    for idx, image in enumerate(tqdm(cluster)):
        image_path = DATA_FOLDER / image.path
        target_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

        source_nas_path = f"/fishsense_data/REEF/data/{image.path}"
        filestation.get_file(source_nas_path, "download", dest_path=str(image_path.parent))

        img = process_raw(image_path)
        img = rectify(img, camera_intrinsics.camera_matrix, camera_intrinsics.distortion_coefficients)

        height, width = img.shape[:2]

        # Define text properties
        text = f"{idx + 1}/{len(cluster)}"
        org = (width - 350, height - 75) # Bottom-right corner of the text
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 5
        color = (0, 0, 255) # Red color (BGR)
        thickness = 10
        lineType = cv2.LINE_AA

        # Put the text on the image
        cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType)

        cv2.imwrite(target_path.as_posix(), img)

        target_nas_path = f"/fishsense_process_work/preprocess_groups_jpeg"
        filestation.upload_file(target_nas_path, str(target_path), overwrite=True)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]



























Upload Progress: 100%|██████████| 7.11M/7.11M [00:06<00:00, 1.23MB/s]



























Upload Progress: 100%|██████████| 7.11M/7.11M [00:08<00:00, 917kB/s]


  0%|          | 0/2 [00:00<?, ?it/s]





























Upload Progress: 100%|██████████| 7.11M/7.11M [00:04<00:00, 1.51MB/s]



























Upload Progress: 100%|██████████| 7.12M/7.12M [00:05<00:00, 1.45MB/s]


...





























Upload Progress: 100%|██████████| 7.36M/7.36M [00:05<00:00, 1.41MB/s]



























Upload Progress: 100%|██████████| 7.37M/7.37M [00:05<00:00, 1.35MB/s]


  0%|          | 0/5 [00:00<?, ?it/s]






























Upload Progress: 100%|██████████| 7.39M/7.39M [00:06<00:00, 1.27MB/s]





























Upload Progress: 100%|██████████| 7.34M/7.34M [00:07<00:00, 1.09MB/s]




























Upload Progress: 100%|██████████| 7.37M/7.37M [00:05<00:00, 1.32MB/s]


























Upload Progress: 100%|██████████| 7.39M/7.39M [00:05<00:00, 1.45MB/s]




























Upload Progress: 100%|██████████| 7.31M/7.31M [00:06<00:00, 1.20MB/s]


  0%|          | 0/3 [00:00<?, ?it/s]





























Upload Progress: 100%|██████████| 7.54M/7.54M [00:06<00:00, 1.24MB/s]




























Upload Progress: 100%|██████████| 7.51M/7.51M [00:05<00:00, 1.37MB/s]





























Upload Progress: 100%|██████████| 7.53M/7.53M [00:06<00:00, 1.29MB/s]
